In [ ]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import math
from ast import literal_eval
from tabulate import tabulate
import os

In [ ]:
soil_dir = '/content/drive/MyDrive/GHG Emissions Paddy Fields Project/Data/soil'
s_bulk_den = nc.Dataset(soil_dir + '/' + 'S_BULK_DEN.nc4')
t_bulk_den = nc.Dataset(soil_dir + '/' + 'T_BULK_DEN.nc4')
s_clay = nc.Dataset(soil_dir + '/' + 'S_CLAY.nc4')
t_clay = nc.Dataset(soil_dir + '/' + 'T_CLAY.nc4')
s_soc = nc.Dataset(soil_dir + '/' + 'S_OC.nc4')
t_soc = nc.Dataset(soil_dir + '/' + 'T_OC.nc4')
s_ph = nc.Dataset(soil_dir + '/' + 'S_PH_H2O.nc4')
t_ph = nc.Dataset(soil_dir + '/' + 'T_PH_H2O.nc4')

In [ ]:
thanjavur = pd.read_csv('/content/drive/MyDrive/GHG Emissions Paddy Fields Project/Data/thanjavur_rice_area.csv')

In [ ]:
def round_nearest(x, base):
    return base * round(x/base)

In [ ]:
def index(bounds):
  bounds = literal_eval(bounds)
  lon_min_i = int((bounds[0]+180)/0.05)
  lat_min_i = int((bounds[1]+90)/0.05)

  return lon_min_i, lat_min_i, lon_min_i+1, lat_min_i+1

In [ ]:
datasets = {'S_BULK_DEN': s_bulk_den, 'T_BULK_DEN': t_bulk_den, 'S_CLAY': s_clay, 'T_CLAY': t_clay,
             'S_OC': s_soc, 'T_OC': t_soc, 'S_PH_H2O': s_ph, 'T_PH_H2O': t_ph}

In [ ]:
for key, ds in datasets.items():
  ds[key][:].data[ds[key][:].data == -1] = np.nan

In [ ]:
def get_data(indices):
  data = []
  for key, ds in datasets.items():
    data.append(ds[key][indices[1]:indices[3]+1, indices[0]:indices[2]+1].data)
  for e in data:
    if (e.size ==0):
      return -1, -1, -1, -1, -1, -1, -1, -1
  bulk_den = np.mean([data[0], data[1]], axis = 0)
  clay = np.mean([data[2], data[3]], axis = 0)
  soc = np.mean([data[4], data[5]], axis = 0)
  ph = np.mean([data[6], data[7]], axis = 0)
  return np.max(bulk_den), np.min(bulk_den), np.max(clay), np.min(clay), np.max(soc), np.min(soc), np.max(ph), np.min(ph)

In [ ]:
def df_cols(bounds):
  return get_data(index(bounds))

In [ ]:
thanjavur['max_den'], thanjavur['min_den'], thanjavur['max_clay'], thanjavur['min_clay'], thanjavur['max_soc'], thanjavur['min_soc'], thanjavur['max_ph'], thanjavur['min_ph'] = zip(*thanjavur['coords'].map(df_cols))

In [ ]:
thanjavur =  thanjavur.loc[:, thanjavur.columns != 'Unnamed: 0']

In [ ]:
cols = thanjavur.columns[1:]
for col in cols:
  thanjavur[col] = thanjavur[col].replace({-1:np.mean(thanjavur[col])})

In [ ]:
thanjavur['min_clay'] = thanjavur['min_clay']/100
thanjavur['max_clay'] = thanjavur['max_clay']/100
thanjavur['min_soc'] = thanjavur['min_soc']/100
thanjavur['max_soc'] = thanjavur['max_soc']/100

In [ ]:
thanjavur['lon'] = thanjavur['coords'].apply(lambda x: literal_eval(x)[0])
thanjavur['lat'] = thanjavur['coords'].apply(lambda x: literal_eval(x)[1])

In [ ]:
def round_down(num, divisor):
      return "{:.2f}".format(num - (num%divisor))

In [ ]:
thanjavur['lat_nearest'] = thanjavur['lat'].apply(lambda x: (round_down(x, 0.25)))
thanjavur['lon_nearest'] = thanjavur['lon'].apply(lambda x: (round_down(x, 0.25)))
thanjavur['climate_file'] = (thanjavur['lat_nearest'] + thanjavur['lon_nearest'])

In [ ]:
thanjavur['climate_file'] = thanjavur['climate_file'].apply(lambda x: x.replace(".", ""))

In [ ]:
thanjavur['id'] = thanjavur.index + 1

In [ ]:
thanjavur['district'] = 'Thanjavur'

In [ ]:
thanjavur['n_dep'] = 0

In [ ]:
climate_soil_df = thanjavur[['id', 'district', 'lon', 'lat', 'climate_file', 'n_dep', 'max_soc', 'min_soc', 'max_clay', 'min_clay', 'max_ph', 'min_ph', 'max_den', 'min_den']]

In [ ]:
climate_soil_df[['slope', 'slainity_index', 'snow', 'column', 'row',' region']] = 0

In [ ]:
def to_fwf(df, fname):
    content = tabulate(df.values.tolist(), tablefmt="tsv", showindex='never', numalign='left')
    f = open(fname, "w")
    f.write('General information of grid cells: location, climate station ID, and soil properies\n')
    f.write(f'20\tCountry\tCountry-name\tLon\tLat\tClimate-file\tN-dep\tSOCmax\tSOCmin\tClaymax\tClaymin\tpHmax\tpHmin\tDensmax\tDensmin\tSlope\tSalinityIndex\tSnow\tColumn\tRow\tRegion\n')
    f.write(f'20\tCountry\tCountry-name\tLon\tLat\tClimate-file\tN-dep\tSOCmax\tSOCmin\tClaymax\tClaymin\tpHmax\tpHmin\tDensmax\tDensmin\tSlope\tSalinityIndex\tSnow\tColumn\tRow\tRegion\n')
    f.write(content)
    f.close()

In [ ]:
for dis in thanjavur.district.unique():
  df = climate_soil_df[climate_soil_df['district'] == dis]
  num_taluk = len(df)
  newpath = f'/content/drive/MyDrive/GHG Emissions Paddy Fields Project/DNDC Input/{dis}'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
  to_fwf(df, f'/content/drive/MyDrive/GHG Emissions Paddy Fields Project/DNDC Input/{dis}/{dis}r_ClimateSoil.txt')